In [ ]:
print("bonjour le monde")

Pouvoir d'arrêt collisionnel des protons:

$$\frac{S_{col}}{\rho}=-\left(\frac{d T}{\rho dx}\right)_{col}=NZ\int_0^{T_e^{max}}T'\left(\frac{d\sigma}{dT'}\right)_{col}d T'$$

$$T_e^{max}=\frac{2m_e c^2(\gamma^2 -1)}{1+2\gamma\frac{m_e}{m_p}+\left(\frac{m_e}{m_p}\right)^2}$$